# Vacation Analysis


In [90]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [92]:
cities_data = "City_data.csv"
cities_df = pd.read_csv(cities_data)
cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hithadhoo,-0.6000,73.0833,82.18,69.0,100.0,5.10,MV,1.611433e+09
1,1,severo-kurilsk,50.6789,156.1250,22.26,95.0,88.0,34.27,RU,1.611433e+09
2,2,puerto ayora,-0.7393,-90.3518,82.40,74.0,40.0,11.50,EC,1.611433e+09
3,3,atuona,-9.8000,-139.0333,78.12,79.0,2.0,18.39,PF,1.611433e+09
4,4,taolanaro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,snasa,64.2457,12.3778,15.04,93.0,100.0,6.64,NO,1.611433e+09
6,6,kapaa,22.0752,-159.3190,72.00,94.0,90.0,20.71,US,1.611433e+09
7,7,clonakilty,51.6231,-8.8706,37.99,100.0,75.0,4.61,IE,1.611433e+09
8,8,upernavik,72.7868,-56.1549,21.18,79.0,100.0,5.50,GL,1.611433e+09
9,9,saint-philippe,-21.3585,55.7679,78.80,73.0,0.0,5.75,RE,1.611433e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [93]:
import gmaps
gmaps.configure(api_key=g_key)

In [108]:
# locations = cities_df[["Lat", "Lng"]].notna()
locations_na=cities_df.dropna()
locations=locations_na[["Lat","Lng"]]
humidity = locations_na['Humidity'].astype(float)

fig = gmaps.figure(center=(0, 0), zoom_level=2.2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [109]:
ideal=locations_na[(locations_na['Wind Speed']<10) & (locations_na['Cloudiness']== 0.0) 
                   & (locations_na['Max Temp']>70) & (locations_na['Max Temp']<80) ]

# ideal

### Hotel Map

* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters. - I had to move to 6000 to get a result for my list
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [110]:
ideal['Hotel Name']=''
# ideal

<ipython-input-110-b49fb59469aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal['Hotel Name']=''


In [111]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius":"6000",
    "type": "lodging",
    "key": g_key,
}

for index, row in ideal.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    location=f'{lat},{lng}'
    
    # add location lat and long to params dict
    params['location'] = location

    # assemble url and make API request

    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        ideal.loc[index, 'Hotel'] = results[0]['name']
               
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
# ideal

C:\Users\deler\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\deler\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [112]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal.iterrows()]
locations = ideal[["Lat", "Lng"]]


In [113]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                             info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))